<a href="https://colab.research.google.com/github/mohansanthichinthala/Artifical-Interface/blob/main/maze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
from collections import deque

class Maze:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        # Create maze grid: 0 = wall, 1 = path
        self.grid = [[0 for _ in range(width)] for _ in range(height)]
        self.start = (1, 1)
        self.end = (height - 2, width - 2)
        self.solution_path = []

    def generate_maze(self):
        """Generate maze using recursive backtracking algorithm"""
        # Start with all walls
        self.grid = [[0 for _ in range(self.width)] for _ in range(self.height)]

        # Stack for backtracking
        stack = []
        # Start position
        start_x, start_y = 1, 1
        self.grid[start_x][start_y] = 1
        stack.append((start_x, start_y))

        directions = [(0, 2), (2, 0), (0, -2), (-2, 0)]

        while stack:
            current_x, current_y = stack[-1]

            # Get unvisited neighbors
            neighbors = []
            for dx, dy in directions:
                nx, ny = current_x + dx, current_y + dy
                if (1 <= nx < self.height - 1 and 1 <= ny < self.width - 1
                    and self.grid[nx][ny] == 0):
                    neighbors.append((nx, ny))

            if neighbors:
                # Choose random neighbor
                next_x, next_y = random.choice(neighbors)

                # Remove wall between current and next cell
                wall_x = current_x + (next_x - current_x) // 2
                wall_y = current_y + (next_y - current_y) // 2

                self.grid[next_x][next_y] = 1
                self.grid[wall_x][wall_y] = 1

                stack.append((next_x, next_y))
            else:
                stack.pop()

        # Ensure start and end points are paths
        self.grid[self.start[0]][self.start[1]] = 1
        self.grid[self.end[0]][self.end[1]] = 1

    def solve_maze_bfs(self):
        """Solve maze using Breadth-First Search"""
        queue = deque([(self.start, [self.start])])
        visited = set([self.start])

        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

        while queue:
            (x, y), path = queue.popleft()

            if (x, y) == self.end:
                self.solution_path = path
                return True

            for dx, dy in directions:
                nx, ny = x + dx, y + dy

                if (0 <= nx < self.height and 0 <= ny < self.width
                    and self.grid[nx][ny] == 1 and (nx, ny) not in visited):
                    visited.add((nx, ny))
                    queue.append(((nx, ny), path + [(nx, ny)]))

        return False

    def solve_maze_dfs(self):
        """Solve maze using Depth-First Search"""
        def dfs(x, y, path, visited):
            if (x, y) == self.end:
                self.solution_path = path
                return True

            directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

            for dx, dy in directions:
                nx, ny = x + dx, y + dy

                if (0 <= nx < self.height and 0 <= ny < self.width
                    and self.grid[nx][ny] == 1 and (nx, ny) not in visited):
                    visited.add((nx, ny))
                    if dfs(nx, ny, path + [(nx, ny)], visited):
                        return True
                    visited.remove((nx, ny))

            return False

        visited = set([self.start])
        return dfs(self.start[0], self.start[1], [self.start], visited)

    def print_maze(self, show_solution=False):
        """Print maze to console"""
        symbols = {
            'wall': '█',
            'path': ' ',
            'start': 'S',
            'end': 'E',
            'solution': '·'
        }

        for i in range(self.height):
            row = ""
            for j in range(self.width):
                if (i, j) == self.start:
                    row += symbols['start']
                elif (i, j) == self.end:
                    row += symbols['end']
                elif show_solution and (i, j) in self.solution_path:
                    row += symbols['solution']
                elif self.grid[i][j] == 1:
                    row += symbols['path']
                else:
                    row += symbols['wall']
            print(row)
        print()

class MazeGame:
    def __init__(self):
        self.maze = None

    def create_new_maze(self, width=21, height=21):
        """Create a new maze with specified dimensions"""
        # Ensure odd dimensions for proper maze generation
        if width % 2 == 0:
            width += 1
        if height % 2 == 0:
            height += 1

        self.maze = Maze(width, height)
        print(f"Generating {height}x{width} maze...")
        self.maze.generate_maze()
        print("Maze generated!")

    def display_maze(self):
        """Display the current maze"""
        if not self.maze:
            print("No maze created yet!")
            return

        print("\nCurrent Maze:")
        print("S = Start, E = End, █ = Wall,   = Path")
        self.maze.print_maze()

    def solve_maze(self, algorithm='bfs'):
        """Solve the maze using specified algorithm"""
        if not self.maze:
            print("No maze created yet!")
            return

        print(f"Solving maze using {algorithm.upper()}...")
        start_time = time.time()

        if algorithm.lower() == 'bfs':
            solved = self.maze.solve_maze_bfs()
        elif algorithm.lower() == 'dfs':
            solved = self.maze.solve_maze_dfs()
        else:
            print("Unknown algorithm! Use 'bfs' or 'dfs'")
            return

        end_time = time.time()

        if solved:
            print(f"Maze solved in {end_time - start_time:.4f} seconds!")
            print(f"Solution path length: {len(self.maze.solution_path)}")
            print("\nMaze with solution (· = solution path):")
            self.maze.print_maze(show_solution=True)
        else:
            print("No solution found!")

    def run_interactive(self):
        """Run interactive maze game"""
        print("=== Python Maze Generator and Solver ===")
        print("Commands:")
        print("1. 'new <width> <height>' - Create new maze")
        print("2. 'show' - Display current maze")
        print("3. 'solve <bfs|dfs>' - Solve maze")
        print("4. 'quit' - Exit")
        print()

        while True:
            try:
                command = input("Enter command: ").strip().lower().split()

                if not command:
                    continue

                if command[0] == 'quit':
                    print("Goodbye!")
                    break

                elif command[0] == 'new':
                    if len(command) == 3:
                        width, height = int(command[1]), int(command[2])
                        self.create_new_maze(width, height)
                    else:
                        self.create_new_maze()

                elif command[0] == 'show':
                    self.display_maze()

                elif command[0] == 'solve':
                    algorithm = command[1] if len(command) > 1 else 'bfs'
                    self.solve_maze(algorithm)

                else:
                    print("Unknown command!")

            except (ValueError, IndexError):
                print("Invalid command format!")
            except KeyboardInterrupt:
                print("\nGoodbye!")
                break

# Demo function
def demo():
    """Run a quick demo of the maze functionality"""
    print("=== Maze Demo ===")

    # Create game instance
    game = MazeGame()

    # Create a small maze
    game.create_new_maze(15, 15)
    game.display_maze()

    # Solve with BFS
    game.solve_maze('bfs')

    # Create a larger maze
    print("\n" + "="*50)
    print("Creating larger maze...")
    game.create_new_maze(25, 25)

    # Compare BFS vs DFS
    game.solve_maze('bfs')
    print("\n" + "-"*30)
    game.solve_maze('dfs')

if __name__ == "__main__":
    # Uncomment one of the following:

    # Run demo
    demo()

    # Or run interactive mode
    # game = MazeGame()
    # game.run_interactive()

=== Maze Demo ===
Generating 15x15 maze...
Maze generated!

Current Maze:
S = Start, E = End, █ = Wall,   = Path
███████████████
█S  █     █   █
███ █ █████ █ █
█   █       █ █
█ ███ ███████ █
█ █     █   █ █
█ █████ █ █ █ █
█     █   █ █ █
█████ █████ █ █
█   █ █   █ █ █
█ ███ █ █ ███ █
█     █ █   █ █
█ █████ ███ █ █
█       █    E█
███████████████

Solving maze using BFS...
Maze solved in 0.0001 seconds!
Solution path length: 45

Maze with solution (· = solution path):
███████████████
█S··█     █   █
███·█ █████ █ █
█···█       █ █
█·███ ███████ █
█·█     █   █ █
█·█████ █ █ █ █
█·····█   █ █ █
█████·█████ █ █
█   █·█···█ █ █
█ ███·█·█·███ █
█·····█·█···█ █
█·█████·███·█ █
█·······█  ··E█
███████████████


Creating larger maze...
Generating 25x25 maze...
Maze generated!
Solving maze using BFS...
Maze solved in 0.0006 seconds!
Solution path length: 165

Maze with solution (· = solution path):
█████████████████████████
█S█      ···············█
█·█████ █·█████████████·█
█·····█ █·····